In [ ]:
!pip install -q torch transformers huggingface_hub datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from getpass import getpass
api_token = getpass("Enter your Wandb api key: ")


Enter your Wandb api key: ··········


In [ ]:
import wandb


In [ ]:
wandb.login(key=api_token)
wandb.init(project='nepbert_finetuning3')


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nadikapoudel16. Use `wandb login --relogin` to force relogin


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,TrainingArguments,Trainer

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
from datasets import load_dataset,DatasetDict
from huggingface_hub import Repository

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('Rajan/NepaliBERT')
model=AutoModelForSequenceClassification.from_pretrained('Rajan/NepaliBERT')

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/987k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rajan/NepaliBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset_file_path='/content/drive/MyDrive/major/cleaned_data_final.xlsx'

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_excel(dataset_file_path)

In [ ]:
df['गुनासो वर्ग'].unique()

array(['अर्थ सबन्धी', 'अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी',
       'कर्मचारी सम्वन्धी', 'खानेपानी', 'निर्माण कार्य सम्बन्धी',
       'पार्किङ्ग तथा मेट्रो पुलिस', 'प्राकृतिक श्रोत/साधन सम्बन्धी',
       'फोहोरमैला व्यवस्थापन', 'लागु पदार्थ सम्बन्धी',
       'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी', 'शान्ति सुरक्षा',
       'सूचना तथा संचार', 'सोधपुछ, सुझाव, प्रशंसा सम्बन्धी',
       'स्वास्थ्यसँग सम्बन्धी'], dtype=object)

In [ ]:
df.head()

,गुनासो वर्ग,गुनासो
0,अर्थ सबन्धी,मीनभवन र तिनकुनेको बीचमा रहेको एलजी शोरुम माथि...
1,अर्थ सबन्धी,- भ्याट बिल जारी गरिएको छैन
2,अर्थ सबन्धी,. जानकारी - अनुरोध\nकुनै पनि मानिसको जागिर अवध...
3,अर्थ सबन्धी,एटीएम कार्ड हराएको लगभग २ वर्ष जति भइसकेको छ न...
4,अर्थ सबन्धी,"वास्तवमा, यो उल्लिखित समस्या स्टक एक्सचेन्जसँग..."


In [ ]:
df['गुनासो वर्ग'].value_counts()

वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी      2849
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी             1883
पार्किङ्ग तथा मेट्रो पुलिस                  1701
कर्मचारी सम्वन्धी                           1337
अर्थ सबन्धी                                  860
खानेपानी                                     800
स्वास्थ्यसँग सम्बन्धी                        694
सूचना तथा संचार                              640
शान्ति सुरक्षा                               634
फोहोरमैला व्यवस्थापन                         630
निर्माण कार्य सम्बन्धी                       605
प्राकृतिक श्रोत/साधन सम्बन्धी                200
लागु पदार्थ सम्बन्धी                         157
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी      71
Name: गुनासो वर्ग, dtype: int64

In [ ]:
counts=df['गुनासो वर्ग'].value_counts()
max_count= max(counts)
max_count

2849

In [ ]:
df_final=pd.DataFrame()
for category,count in counts.items():
  df_cat=df[df['गुनासो वर्ग']==category]
  if count < max_count:
    df_cat_oversampled=df_cat.sample(max_count,replace=True)
    df_cat_oversampled=df_cat_oversampled.reset_index(drop=True)
  else:
    df_cat_oversampled=df_cat

  df_final=pd.concat([df_final,df_cat_oversampled])


In [ ]:
df_final['गुनासो वर्ग'].value_counts()

वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी      2849
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी             2849
पार्किङ्ग तथा मेट्रो पुलिस                  2849
कर्मचारी सम्वन्धी                           2849
अर्थ सबन्धी                                 2849
खानेपानी                                    2849
स्वास्थ्यसँग सम्बन्धी                       2849
सूचना तथा संचार                             2849
शान्ति सुरक्षा                              2849
फोहोरमैला व्यवस्थापन                        2849
निर्माण कार्य सम्बन्धी                      2849
प्राकृतिक श्रोत/साधन सम्बन्धी               2849
लागु पदार्थ सम्बन्धी                        2849
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी    2849
Name: गुनासो वर्ग, dtype: int64

In [ ]:
df_final.sample(5)

,गुनासो वर्ग,गुनासो
1305,प्राकृतिक श्रोत/साधन सम्बन्धी,राजस्व चुहावट डुबान कटान मानबिय क्षति समेत हुन...
7797,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,सर नमस्ते मैले बिगत ३ हप्ता देखी हामी दुई बूढ़...
678,"सोधपुछ, सुझाव, प्रशंसा सम्बन्धी",मेरो pan no 113902204 हो तर आन्तरिक राजस्व बिभ...
1054,प्राकृतिक श्रोत/साधन सम्बन्धी,शिबपुरी राष्ट्रिय निकुन्जको तटिय क्षेत्र भित्र...
2611,स्वास्थ्यसँग सम्बन्धी,१० वडा वानेश्वरमा १०/१५ कुकुरहरुले भुकेर राती ...


In [ ]:
# Data Preprocessing
nepali_stopwords = set(stopwords.words('nepali'))
english_stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[#\\/।(),०-९<<?!,—–’‘:\u200d]', '', text)
    text = text.strip('"')

    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in nepali_stopwords and word.lower() not in english_stopwords]
    processed_text = ' '.join(filtered_words)
    return processed_text



In [ ]:
df_final['गुनासो'] = df_final['गुनासो'].apply(preprocess_text)

In [ ]:

df_final['गुनासो वर्ग'].value_counts()

वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी      2849
सोधपुछ, सुझाव, प्रशंसा सम्बन्धी             2849
पार्किङ्ग तथा मेट्रो पुलिस                  2849
कर्मचारी सम्वन्धी                           2849
अर्थ सबन्धी                                 2849
खानेपानी                                    2849
स्वास्थ्यसँग सम्बन्धी                       2849
सूचना तथा संचार                             2849
शान्ति सुरक्षा                              2849
फोहोरमैला व्यवस्थापन                        2849
निर्माण कार्य सम्बन्धी                      2849
प्राकृतिक श्रोत/साधन सम्बन्धी               2849
लागु पदार्थ सम्बन्धी                        2849
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी    2849
Name: गुनासो वर्ग, dtype: int64

In [ ]:
label_encoder = LabelEncoder()
df_final['encoded_labels'] = label_encoder.fit_transform(df_final['गुनासो वर्ग'])

In [ ]:
df_final.head()

,गुनासो वर्ग,गुनासो,encoded_labels
6361,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अछाम जिल्ला रेकर्ड भन्छ,9
6362,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अझ यसमा लाईसेन्सको थप्न पाए हुन्थ्यो,9
6363,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अपडेट नभएको सर नागरिक apps माथी उल्लेखित न.हान...,9
6364,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अञ्चल लेख्न लेख्नी बाघ मति मात्रै आउनुपर्ने,9
6365,वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी,अधिकृत छैठौको भोली फम भर्ने अन्तिम मिति हिजो क...,9


In [ ]:
df_final.tail()

,गुनासो वर्ग,गुनासो,encoded_labels
2844,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,मैले पहिलाको श्रीमतिलाई कानुन मालपोतबाट अन्स ब...,1
2845,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,अशन कमलाक्षिमा फुटपाथ पसलेबाट नगर प्रहरीहरुले ...,1
2846,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,बिद्युत विकास बिभाग बाताबरण शाखा निजि क्षेत्र ...,1
2847,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,प्रहरी चौकी कुशेश्वर दुम्जाको प्रहरी हरुले बिप...,1
2848,अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी,नेपाल भारत सीमावर्ती क्षेत्रमा खुलेआम रूपमा रक...,1


In [ ]:
encodings = tokenizer(df_final['गुनासो'].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
train_indices, val_indices = train_test_split(np.arange(len(df_final)), test_size=0.2, stratify=df_final['encoded_labels'])


In [ ]:
# Custom dataset class
class NepaliComplaintsDataset(Dataset):
    def __init__(self, encodings, labels, indices):
        self.encodings = {key: val[indices] for key, val in encodings.items()}
        self.labels = labels[indices]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Creating datasets
train_dataset = NepaliComplaintsDataset(encodings, np.array(df_final['encoded_labels']), train_indices)
val_dataset = NepaliComplaintsDataset(encodings, np.array(df_final['encoded_labels']), val_indices)

In [ ]:
# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
num_labels = len(np.unique(df_final['encoded_labels']))


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("Rajan/NepaliBERT", num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rajan/NepaliBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



In [ ]:


!huggingface-cli whoami

nadika


In [ ]:
training_args = TrainingArguments(
    output_dir='nepali_complaints_classification_nepbert3',          # Directory for saving model and outputs
    num_train_epochs=5,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=50,                # Number of warmup steps
    weight_decay=0.01,               # Weight decay rate #l2 regularization AdamW
    report_to = "wandb",
    logging_dir='./logs',            # Directory for logging
    evaluation_strategy="steps",     # Evaluate at the end of every epoch
    save_steps = 500,
    eval_steps=500,
    logging_steps=500,          # Save model at the end of every epoch
    learning_rate=2e-5,
    load_best_model_at_end=True,     # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Metric to optimize over
    save_total_limit=5,              # Limit the total number of checkpoints to save (useful for saving disk space)
    push_to_hub=True,
    # hub_model_id=hub_model_id,
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    predictions = p.predictions.argmax(axis=1)
    label_ids = p.label_ids
    accuracy = accuracy_score(label_ids, predictions)
    f1 = f1_score(label_ids, predictions, average='weighted')

    return {"accuracy": accuracy, "f1-score": f1}



In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

<ipython-input-29-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1-score
500,1.492100,0.864195,0.723490,0.714339
1000,0.778100,0.624062,0.797443,0.792301
1500,0.586500,0.534205,0.824267,0.812536
2000,0.462500,0.425015,0.857608,0.855304
2500,0.364800,0.385573,0.875909,0.872476
3000,0.300100,0.342368,0.889947,0.889120
3500,0.272300,0.319915,0.900727,0.898120
4000,0.253800,0.289768,0.908498,0.906638
4500,0.231000,0.267575,0.920281,0.918922
5000,0.147800,0.302940,0.921033,0.918669


<ipython-input-29-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-29-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-29-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-29-1529dc6678a3>:8: UserWarning: To copy construct from a 

TrainOutput(global_step=11220, training_loss=0.27445745570136904, metrics={'train_runtime': 3181.5466, 'train_samples_per_second': 56.414, 'train_steps_per_second': 3.527, 'total_flos': 2.378099916321792e+16, 'train_loss': 0.27445745570136904, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/nadika/nepali_complaints_classification_nepbert3/commit/c9f3b6192751773dadd95a6218b3a54012a27a52', commit_message='End of training', commit_description='', oid='c9f3b6192751773dadd95a6218b3a54012a27a52', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.evaluate()

<ipython-input-29-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.24990147352218628,
 'eval_accuracy': 0.9453497117071948,
 'eval_f1-score': 0.944329069963225,
 'eval_runtime': 20.0555,
 'eval_samples_per_second': 198.898,
 'eval_steps_per_second': 12.465,
 'epoch': 5.0}

In [ ]:
tokenizer.push_to_hub("nadika/nepali_complaints_classification_nepbert3")


README.md:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nadika/nepali_complaints_classification_nepbert3/commit/45e36a2d88af6c06fa0639fc7821ce2c288baa29', commit_message='Upload tokenizer', commit_description='', oid='45e36a2d88af6c06fa0639fc7821ce2c288baa29', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
labels = label_encoder.classes_  # This gets the ordered list of class names
id2label = dict(enumerate(labels))
id2label

{0: 'अर्थ सबन्धी',
 1: 'अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी',
 2: 'कर्मचारी सम्वन्धी',
 3: 'खानेपानी',
 4: 'निर्माण कार्य सम्बन्धी',
 5: 'पार्किङ्ग तथा मेट्रो पुलिस',
 6: 'प्राकृतिक श्रोत/साधन सम्बन्धी',
 7: 'फोहोरमैला व्यवस्थापन',
 8: 'लागु पदार्थ सम्बन्धी',
 9: 'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी',
 10: 'शान्ति सुरक्षा',
 11: 'सूचना तथा संचार',
 12: 'सोधपुछ, सुझाव, प्रशंसा सम्बन्धी',
 13: 'स्वास्थ्यसँग सम्बन्धी'}

In [ ]:
label2id = {label: id for id, label in id2label.items()}
label2id

{'अर्थ सबन्धी': 0,
 'अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी': 1,
 'कर्मचारी सम्वन्धी': 2,
 'खानेपानी': 3,
 'निर्माण कार्य सम्बन्धी': 4,
 'पार्किङ्ग तथा मेट्रो पुलिस': 5,
 'प्राकृतिक श्रोत/साधन सम्बन्धी': 6,
 'फोहोरमैला व्यवस्थापन': 7,
 'लागु पदार्थ सम्बन्धी': 8,
 'वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी': 9,
 'शान्ति सुरक्षा': 10,
 'सूचना तथा संचार': 11,
 'सोधपुछ, सुझाव, प्रशंसा सम्बन्धी': 12,
 'स्वास्थ्यसँग सम्बन्धी': 13}

In [ ]:
model.save_pretrained('./my_nepalibert_model')
tokenizer.save_pretrained('./my_nepalibert_model')

('./my_nepalibert_model/tokenizer_config.json',
 './my_nepalibert_model/special_tokens_map.json',
 './my_nepalibert_model/vocab.txt',
 './my_nepalibert_model/added_tokens.json',
 './my_nepalibert_model/tokenizer.json')

In [ ]:
model_name="./my_nepalibert_model"

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_name)


In [ ]:
config.id2label = id2label
config.label2id = label2id

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [ ]:
config.save_pretrained(model_name)

In [ ]:
repo_name="nadika/nepali_complaints_classification_nepbert3"

In [ ]:
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/nadika/nepali_complaints_classification_nepbert3/commit/ca17a165b5654d59d37e5035ab4589f827d95a1b', commit_message='Upload tokenizer', commit_description='', oid='ca17a165b5654d59d37e5035ab4589f827d95a1b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_id = "nadika/nepali_complaints_classification_nepbert3"  # replace with your model's repository name
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [ ]:


def preprocess_and_tokenize(text):
    processed_text = preprocess_text(text)  # Use your preprocessing here
    encoded_input = tokenizer(processed_text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    return encoded_input

def predict(text):
    # Preprocess and tokenize the text
    encoded_input = preprocess_and_tokenize(text)

    # Generate predictions
    with torch.no_grad():
        output = model(**encoded_input)

    # The output is logits; apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(output.logits, dim=-1)

    # Convert probabilities to numpy for easier handling
    probabilities = probabilities.detach().cpu().numpy()

    # Get the most likely class index
    predicted_class_idx = probabilities.argmax()

    # Decode the predicted class index to the class name using the model's config
    predicted_class = model.config.id2label[predicted_class_idx]
    return predicted_class, probabilities[0][predicted_class_idx]




In [ ]:
# Example text for prediction
new_text = "यस महानगरपालिकाको सुनाकोठी नखिपोट नख्खुडोल गोकुल आवास कान्तिपुर कोलोनीका वासिन्दाले दिनरात यो धुंवाको सास कहिलेसम्म फेर्नुपर्ने हो हजुर?"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी, Probability: 0.7418889999389648


In [ ]:
# Example text for prediction
new_text = "सुनसरी जिल्ला इटहरी उप महानगर भित्र अवस्थित धरान पुग्ने यो बाटो । कुन ठेकेदारले यो काम गर्दा कति पैसा कुम्ल्याउन पाइने हो र आफ्नो निर्वाचन ताका खर्च भएको करोड उठाउन पाइन्छ भनेर त हैन ?"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: सोधपुछ, सुझाव, प्रशंसा सम्बन्धी, Probability: 0.9648656845092773


In [ ]:
# Example text for prediction
new_text = "इटहरीदेखि पूर्वाञ्चल विश्वविद्यालय, माेरङ हुँदै पूर्व जाने बाटाे जुन लालभित्तीमा गएर टुङ्गिन्छ, पिच गर्नुपर्‍याे। बीचमा दुइटा ठूला खाेला छन्, त्यसमा पुल हाल्नुपर्‍याे। याे बाटाेलाई वैकल्पिक राजमार्गकाे रुपमा विकास गर्नुप्रयो।"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: सोधपुछ, सुझाव, प्रशंसा सम्बन्धी, Probability: 0.7649959921836853


In [ ]:
# Example text for prediction
new_text = "मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग। बाह्रबिसेका मेयरको मनोमानी- दोहोरीमा रमाइलो गरेको बिलसमेत नगरपालिकाबाटै भुक्तानी गर्न दबाब मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग।"
predicted_class, probability = predict(new_text)
print(f"Predicted Class: {predicted_class}, Probability: {probability}")

Predicted Class: अर्थ सबन्धी, Probability: 0.6409968733787537


In [ ]:
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
predictions = []
labels = []

with torch.no_grad():
    for batch in val_loader:
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        model.to(device)
        output = model(inputs, attention_mask=attention_mask)
        logits = output.logits.to(device)
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(preds)
        labels.extend(batch['labels'].cpu().numpy())

<ipython-input-29-1529dc6678a3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Calculate accuracy
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
print(f"Precision: {precision}\nRecall: {recall}\nF1 Score: {f1}")


Accuracy: 0.9453497117071948
Precision: 0.9444476418760597
Recall: 0.9453497117071948
F1 Score: 0.944329069963225
